In [1]:
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

dfs = defaultdict(lambda: defaultdict(dict))

for n_folds in [3,5]:
    for tuning_setting in ['fs', 'of']:
        for seed in [1,42]:
            try:
                dfs[n_folds][tuning_setting][seed] = pd.read_csv(f'data/dml_summary_{n_folds}folds_{tuning_setting}_seed{seed}.csv')
                dfs[n_folds][tuning_setting][seed]['tuning'] = np.where(dfs[n_folds][tuning_setting][seed]['tuning']=='tune', tuning_setting, 'no_tune')
            except FileNotFoundError:
                print(f'File dml_summary_{n_folds}folds_{tuning_setting}_seed{seed}.csv not found, skipping.')

In [3]:
dfs[n_folds][tuning_setting][seed]

,tuning,model,coef_d,std err_d,t_d,P>|t|_d,2.5 %_d,97.5 %_d
0,no_tune,lasso,0.492841,1.484517,0.331988,0.739899,-2.416759,3.402442
1,no_tune,rf,0.626062,0.426000,1.469630,0.141662,-0.208882,1.461007
2,no_tune,xgb,41.423258,75.148092,0.551222,0.581482,-105.864296,188.710812
3,of,lasso,0.337591,0.317974,1.061692,0.288376,-0.285628,0.960809
4,of,rf,0.665796,0.397762,1.673854,0.094159,-0.113804,1.445396
5,of,xgb,0.410335,0.749654,0.547365,0.584128,-1.058961,1.879630


In [4]:
all_df_list = []

for nfold in dfs:
    for setting in dfs[nfold]:
        for seed in dfs[nfold][setting]:
            dfs[nfold][setting][seed]['seed'] = seed
            dfs[nfold][setting][seed]['nfold'] = nfold
            all_df_list.append(dfs[nfold][setting][seed])

full_df = pd.concat(all_df_list, ignore_index=True)

In [5]:
full_df = full_df.drop_duplicates(subset=[col for col in full_df.columns if col != 'tuning_setting'])

In [ ]:
full_df_42 = full_df[full_df['seed']==42]

In [7]:
full_df_42 = full_df_42[["tuning","model","coef_d","std err_d","nfold",'P>|t|_d']]

In [8]:
summary = full_df_42.pivot_table(
    index=['nfold', 'model'],
    columns='tuning',
    values=['coef_d', 'std err_d','P>|t|_d'],
)

summary.columns = [f'{stat}_{tune}' for stat, tune in summary.columns]
summary = summary.reset_index()


In [11]:
summary[summary['nfold']==3]

,nfold,model,P>|t|_d_fs,P>|t|_d_no_tune,P>|t|_d_of,coef_d_fs,coef_d_no_tune,coef_d_of,std err_d_fs,std err_d_no_tune,std err_d_of
0,3,lasso,0.319076,0.382930,0.514624,0.302598,-1.759336,0.199957,0.303704,2.016409,0.306845
1,3,rf,0.513001,0.453843,0.611197,0.242183,0.291343,0.177235,0.370213,0.388963,0.348636
2,3,xgb,0.766890,0.484588,0.844115,0.182280,-57.074355,-0.158705,0.614888,81.658142,0.807117
